# Procesamiento de lenguaje natural
## Entrega n°2
### Ramiro Andrés Feichubuinm



### Objetivo
    -Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
    -Probar términos de interés y explicar similitudes en el espacio de embeddings.
    -Intentar plantear y probar tests de analogías.
    -Graficar los embeddings resultantes.
    -Sacar conclusiones.

Directivas necesarias por temas de compatibilidad entre versiones:

In [1]:
!pip uninstall -y numpy gensim pandas
!pip install --upgrade numpy gensim pandas==2.2.2

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: gensim 4.3.3
Uninstalling gensim-4.3.3:
  Successfully uninstalled gensim-4.3.3
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.0 MB)
Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylin

In [2]:
!pip install tensorflow

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
from gensim.models import Word2Vec

In [4]:
import random
from gensim.utils import simple_preprocess
import requests

### Datos
Utilizo como dataset los siguientes tres textos:

In [5]:
import os
import platform

dataset_dir = 'dataset'
os.makedirs(dataset_dir, exist_ok=True)

url_nietzsche = "https://raw.githubusercontent.com/ramafeichu/natural-language-processing/main/desafios/desafio-1/src/Friedrich_Nietzsche_-_Mas_Alla_del_Bien_y_del_Mal.txt"
url_platon = "https://raw.githubusercontent.com/ramafeichu/natural-language-processing/main/desafios/desafio-1/src/Platon_-_El_Banquete.txt"
url_sun_tzu = "https://raw.githubusercontent.com/ramafeichu/natural-language-processing/main/desafios/desafio-1/src/Sun_Tzu_-_El_Arte_de_la_Guerra.txt"

print("Archivos descargados en la carpeta 'dataset'")

if platform.system() == 'Windows':
    os.system(f"curl -o {dataset_dir}/nietzsche.txt \"{url_nietzsche}\"")
    os.system(f"curl -o {dataset_dir}/platon.txt \"{url_platon}\"")
    os.system(f"curl -o {dataset_dir}/sun_tzu.txt \"{url_sun_tzu}\"")
else:
    os.system(f"wget -P {dataset_dir}/ \"{url_nietzsche}\"")
    os.system(f"wget -P {dataset_dir}/ \"{url_platon}\"")
    os.system(f"wget -P {dataset_dir}/ \"{url_sun_tzu}\"")

Archivos descargados en la carpeta 'dataset'


In [6]:
os.listdir("./dataset/")

['Platon_-_El_Banquete.txt',
 'Sun_Tzu_-_El_Arte_de_la_Guerra.txt.1',
 'Sun_Tzu_-_El_Arte_de_la_Guerra.txt',
 'Platon_-_El_Banquete.txt.1',
 'Friedrich_Nietzsche_-_Mas_Alla_del_Bien_y_del_Mal.txt.1',
 'Friedrich_Nietzsche_-_Mas_Alla_del_Bien_y_del_Mal.txt']

In [7]:
ruta = "./dataset/"
archivos = os.listdir("./dataset/")


dfs = []

for archivo in archivos:
    df_temp = pd.read_csv(os.path.join(ruta, archivo), sep='/n', header=None, names=["Texto"], encoding="latin")

    # Separo por punto también
    df_temp = df_temp["Texto"].str.split(r"\.", expand=True).stack().reset_index(drop=True).to_frame()

    df_temp["Fuente"] = archivo
    dfs.append(df_temp)

# Uno todo
df_final = pd.concat(dfs, ignore_index=True)

df_final.rename(columns={0: "Oración"}, inplace=True)

df_final.head()

<ipython-input-7-169964d14922>:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_temp = pd.read_csv(os.path.join(ruta, archivo), sep='/n', header=None, names=["Texto"], encoding="latin")
<ipython-input-7-169964d14922>:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_temp = pd.read_csv(os.path.join(ruta, archivo), sep='/n', header=None, names=["Texto"], encoding="latin")
<ipython-input-7-169964d14922>:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid thi

,Oración,Fuente
0,El Banquete,Platon_-_El_Banquete.txt
1,Platón,Platon_-_El_Banquete.txt
2,El banquete,Platon_-_El_Banquete.txt
3,APOLODOROS,Platon_-_El_Banquete.txt
4,- Creo que estoy bastante bien preparado para ...,Platon_-_El_Banquete.txt


Elimino documentos con menos de tres palabras.

In [8]:
df_final = df_final[df_final["Oración"].str.split().str.len() >= 3]
df_final = df_final.reset_index(drop=True)
df_final.head()

,Oración,Fuente
0,- Creo que estoy bastante bien preparado para ...,Platon_-_El_Banquete.txt
1,"Apolodoros, me dijo, te buscaba precisamente",Platon_-_El_Banquete.txt
2,Quería preguntarte lo que pasó en la casa de ...,Platon_-_El_Banquete.txt
3,Se dice que toda la conversación versó sobre ...,Platon_-_El_Banquete.txt
4,Algo de ello he sabido por un hombre al que P...,Platon_-_El_Banquete.txt


In [9]:
print("Cantidad de documentos:", df_final.shape[0])

Cantidad de documentos: 6346


In [10]:
df_final

,Oración,Fuente
0,- Creo que estoy bastante bien preparado para ...,Platon_-_El_Banquete.txt
1,"Apolodoros, me dijo, te buscaba precisamente",Platon_-_El_Banquete.txt
2,Quería preguntarte lo que pasó en la casa de ...,Platon_-_El_Banquete.txt
3,Se dice que toda la conversación versó sobre ...,Platon_-_El_Banquete.txt
4,Algo de ello he sabido por un hombre al que P...,Platon_-_El_Banquete.txt
...,...,...
6341,Meditó sobre las consecuencias del triunfo del...,Friedrich_Nietzsche_-_Mas_Alla_del_Bien_y_del_...
6342,Si bien hay quienes sostienen que la caracterí...,Friedrich_Nietzsche_-_Mas_Alla_del_Bien_y_del_...
6343,Nietzsche recibió amplio reconocimiento durant...,Friedrich_Nietzsche_-_Mas_Alla_del_Bien_y_del_...
6344,Su influencia fue particularmente notoria en ...,Friedrich_Nietzsche_-_Mas_Alla_del_Bien_y_del_...


### 1 - Preprocesamiento

Utilizo simple_preprocess de gensim ya que divide en palabras, eliminando signos de puntuación.

In [11]:
from gensim.utils import simple_preprocess

sentence_tokens = []

for _, row in df_final.iterrows():
    sentence_tokens.append(simple_preprocess(row[0]))  # Uso simple_preprocess

<ipython-input-11-2ca67eb3dc4a>:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentence_tokens.append(simple_preprocess(row[0]))  # Uso simple_preprocess


In [12]:
# Demos un vistazo
sentence_tokens[:4]

[['creo',
  'que',
  'estoy',
  'bastante',
  'bien',
  'preparado',
  'para',
  'narraros',
  'lo',
  'que',
  'me',
  'pedís',
  'porque',
  'últimamente',
  'cuando',
  'desde',
  'mi',
  'casa',
  'de',
  'faleron',
  'regresaba',
  'la',
  'ciudad',
  'me',
  'vio',
  'un',
  'conocido',
  'mío',
  'que',
  'iba',
  'detrás',
  'de',
  'mí',
  'me',
  'llamó',
  'desde',
  'lejos',
  'bromeando',
  'hombre',
  'de',
  'faleron',
  'apolodoros',
  'no',
  'puedes',
  'acortar',
  'el',
  'paso',
  'me',
  'detuve',
  'lo',
  'esperé'],
 ['apolodoros', 'me', 'dijo', 'te', 'buscaba', 'precisamente'],
 ['quería',
  'preguntarte',
  'lo',
  'que',
  'pasó',
  'en',
  'la',
  'casa',
  'de',
  'agatón',
  'el',
  'día',
  'en',
  'que',
  'cenaron',
  'allí',
  'sócrates',
  'alcibíades',
  'algunos',
  'otros'],
 ['se',
  'dice',
  'que',
  'toda',
  'la',
  'conversación',
  'versó',
  'sobre',
  'el',
  'amor']]

### 2 - Crear los vectores (word2vec)

In [13]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

Utilizo embeddings de menor dimensionalidad al usado en clase.

In [14]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=200,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [15]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [16]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 6346


In [17]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 3703


### 3 - Entrenar embeddings

Incremento el número de epochs.

In [18]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 1597494.375
Loss after epoch 1: 1107939.125
Loss after epoch 2: 1037836.25
Loss after epoch 3: 985473.75
Loss after epoch 4: 938985.5
Loss after epoch 5: 917471.0
Loss after epoch 6: 894486.0
Loss after epoch 7: 865029.0
Loss after epoch 8: 803920.0
Loss after epoch 9: 782086.0
Loss after epoch 10: 762691.0
Loss after epoch 11: 746112.0
Loss after epoch 12: 730872.0
Loss after epoch 13: 715734.0
Loss after epoch 14: 704720.0
Loss after epoch 15: 691430.0
Loss after epoch 16: 684699.0
Loss after epoch 17: 675274.0
Loss after epoch 18: 667047.0
Loss after epoch 19: 638200.0
Loss after epoch 20: 581042.0
Loss after epoch 21: 577666.0
Loss after epoch 22: 570876.0
Loss after epoch 23: 570574.0
Loss after epoch 24: 564272.0
Loss after epoch 25: 564028.0
Loss after epoch 26: 559672.0
Loss after epoch 27: 553868.0
Loss after epoch 28: 552238.0
Loss after epoch 29: 548410.0
Loss after epoch 30: 548520.0
Loss after epoch 31: 545932.0
Loss after epoch 32: 540708.0
Loss after 

(6186545, 9959800)

### 4 - Ensayar

In [19]:
vocabulario = list(w2v_model.wv.key_to_index.keys())
print(vocabulario)

['de', 'que', 'la', 'en', 'el', 'los', 'no', 'es', 'se', 'un', 'una', 'lo', 'más', 'por', 'su', 'las', 'con', 'del', 'como', 'para', 'al', 'si', 'todo', 'ha', 'cuando', 'sus', 'pero', 'ser', 'hombre', 'son', 'sí', 'mismo', 'también', 'amor', 'qué', 'le', 'esto', 'sobre', 'me', 'hasta', 'nosotros', 'sin', 'todos', 'hombres', 'hay', 'bien', 'él', 'ya', 'entre', 'porque', 'ni', 'así', 'tiene', 'ellos', 'vez', 'ese', 'toda', 'algo', 'sólo', 'cosas', 'ahora', 'cual', 'tan', 'este', 'yo', 'nos', 'manera', 'está', 'pues', 'modo', 'incluso', 'siempre', 'voluntad', 'puede', 'moral', 'decir', 'esa', 'sócrates', 'muy', 'contra', 'vida', 'tal', 'sea', 'otro', 'hoy', 'desde', 'espíritu', 'mundo', 'tiempo', 'nada', 'todas', 'fuerza', 'especie', 'tanto', 'ella', 'sino', 'alma', 'hecho', 'todavía', 'esta', 'han', 'parte', 'cosa', 'dios', 'mi', 'te', 'aquí', 'quien', 'uno', 'hacer', 'mejor', 'verdad', 'les', 'mujer', 'hace', 'enemigo', 'misma', 'hacia', 'sido', 'gran', 'otros', 'nuestra', 'terreno', 'a

In [20]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["dinero"], topn=10)

[('perspicacia', 0.5448550581932068),
 ('paciencia', 0.5437718033790588),
 ('emplea', 0.49166959524154663),
 ('tomas', 0.4717920124530792),
 ('cesar', 0.4618047773838043),
 ('caer', 0.4585486650466919),
 ('fecundos', 0.44971203804016113),
 ('entusiasta', 0.435783714056015),
 ('echar', 0.4348371624946594),
 ('empiezan', 0.43426331877708435)]

In [21]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["cristiana"], topn=10)

[('así', 0.050296150147914886),
 ('conocer', 0.033056266605854034),
 ('últimas', 0.027654705569148064),
 ('tendrán', 0.021813642233610153),
 ('entonces', 0.020451411604881287),
 ('campaña', 0.009309171698987484),
 ('cuenta', 0.005527005530893803),
 ('acerca', 0.001309861778281629),
 ('seguir', -0.0028594934847205877),
 ('palabras', -0.005209891125559807)]

In [22]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["ciencias"], topn=10)

[('extensión', 0.5438839793205261),
 ('señales', 0.5422955751419067),
 ('fatigas', 0.5176714658737183),
 ('ocupaciones', 0.5139970779418945),
 ('ocultas', 0.5028538107872009),
 ('divinas', 0.49800944328308105),
 ('orgánicas', 0.4836941361427307),
 ('religiosas', 0.4829261898994446),
 ('elevadas', 0.48077335953712463),
 ('bellas', 0.47957396507263184)]

In [33]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["trabajadores"], topn=10)

[('filosóficos', 0.5485836863517761),
 ('derrotados', 0.5347532629966736),
 ('lento', 0.5192877054214478),
 ('descubrimientos', 0.5088497996330261),
 ('aptos', 0.5061259865760803),
 ('enfermos', 0.5003628730773926),
 ('servidores', 0.48878902196884155),
 ('ignoran', 0.48024213314056396),
 ('primeros', 0.4797508716583252),
 ('confundir', 0.4746837019920349)]

In [24]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["amor"], topn=10)

[('filosofa', 0.4219493567943573),
 ('marte', 0.40788111090660095),
 ('desear', 0.3956020772457123),
 ('inmortalidad', 0.3870488405227661),
 ('porvenir', 0.38544631004333496),
 ('bello', 0.3822583556175232),
 ('unido', 0.380338191986084),
 ('atrevo', 0.37355655431747437),
 ('graves', 0.3732393682003021),
 ('dioniso', 0.3696787655353546)]

In [25]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["filosofía"], topn=10)

[('psicólogos', 0.3729061782360077),
 ('dogmática', 0.37032943964004517),
 ('richard', 0.36683133244514465),
 ('repercusión', 0.3664717972278595),
 ('luna', 0.35660916566848755),
 ('locura', 0.3536168038845062),
 ('ética', 0.35028690099716187),
 ('gigantesca', 0.35019341111183167),
 ('hipótesis', 0.34945276379585266),
 ('histórica', 0.34721341729164124)]

In [26]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["diedaa"])

KeyError: "Key 'diedaa' not present in vocabulary"

In [27]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("amor")
print(vector_love)

[ 4.42739725e-02  7.27622330e-01 -6.56192839e-01 -2.57029176e-01
  4.27362561e-01 -1.20748989e-01 -1.54156461e-01  6.75508454e-02
 -3.31059128e-01 -2.56771594e-01 -8.22252870e-01 -5.60161732e-02
 -8.00501034e-02  4.32416983e-02  3.20488930e-01 -1.34500295e-01
  1.20256044e-01 -2.70081133e-01  6.17090240e-03 -1.50420964e-02
  2.92279292e-02  8.62730965e-02 -2.77690440e-01 -1.28717748e-02
  1.15688458e-01  2.86967844e-01 -5.23339927e-01 -8.86008963e-02
  1.07192388e-02 -2.95371413e-01  6.12847954e-02 -6.74593449e-01
 -9.60733742e-02 -4.28698421e-01 -1.83260128e-01  2.93947071e-01
 -6.93581253e-02 -5.74513137e-01 -1.60835534e-01 -2.37855375e-01
  3.38595957e-01  3.58535558e-01 -3.71404827e-01 -2.52312899e-01
 -2.57040001e-02 -4.45423089e-02 -1.06725669e+00 -5.19868076e-01
 -7.96640038e-01  2.22214878e-01  1.40375778e-01 -2.40627065e-01
 -5.10771051e-02  3.22579056e-01  4.18068975e-01  3.61968368e-01
 -4.79123980e-01 -7.98871040e-01 -1.82106540e-01  2.29816109e-01
 -6.38769865e-02 -5.70953

In [28]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('amor', 1.0),
 ('filosofa', 0.4219493865966797),
 ('marte', 0.40788114070892334),
 ('desear', 0.3956020772457123),
 ('inmortalidad', 0.3870488703250885),
 ('porvenir', 0.38544631004333496),
 ('bello', 0.3822583556175232),
 ('unido', 0.3803381323814392),
 ('atrevo', 0.373556524515152),
 ('graves', 0.3732393682003021)]

## Conclusiones

Por tratarse de un corpus con muy pocos documentos, los resultados son muy dependientes del preprocesamiento y de la forma de entrenar al modelo.

No obstante, el análisis de palabras más y menos relacionadas en el espacio de embeddings revela algunos patrones interesantes sobre cómo el modelo ha capturado relaciones semánticas en los datos de entrenamiento. Algunos ejemplos:

- "dinero":
Se observan términos como "emplea", "fondos", "césar", "echar", y "capital", que podrían estar asociados con aspectos financieros, económicos o decisiones estratégicas. Sin embargo, algunas palabras como "perspicacia" y "paciencia" podrían sugerir un contexto donde el dinero se asocia con cualidades personales o habilidades necesarias para manejarlo.

- "cristiana":
Aquí se encuentran términos como "así", "conocer", "últimas", "campaña", y "acerca". Estas parecen ser palabras más neutras o genéricas, sin una fuerte carga semántica vinculada al término "cristiana", lo que indica que en el espacio vectorial no tienen una conexión relevante con conceptos religiosos.

- "ciencias":
Los términos más similares incluyen "extensión", "señales", "fatigas", "divinas", "orgánicas", y "religiosas". Es interesante notar la mezcla de términos científicos y algunos con connotaciones filosóficas o espirituales, lo que sugiere que en el corpus utilizado, la ciencia podría estar relacionada con cuestiones humanísticas o metafísicas.

- "trabajadores":
Aquí aparecen palabras como "filosóficos", "derrotados", "lento", "enfermos", "servidores", y "ignoran". Algunos de estos términos podrían sugerir un contexto de lucha laboral, condiciones laborales difíciles o reflexiones sobre la situación de los trabajadores en el corpus analizado.

### 5 - Visualizar agrupación de vectores




In [29]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [30]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

Exception ignored on calling ctypes callback function: <function ThreadpoolController._find_libraries_with_dl_iterate_phdr.<locals>.match_library_callback at 0x7bd0a9ee5e40>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1005, in match_library_callback
    self._make_controller_from_path(filepath)
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1187, in _make_controller_from_path
    lib_controller = controller_class(
                     ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 114, in __init__
    self.dynlib = ctypes.CDLL(filepath, mode=_RTLD_NOLOAD)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: dlopen() error


In [31]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [32]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### 6 - Test de Analogías


In [47]:
print(w2v_model.wv.most_similar(positive=["dinero", "trabajo"], negative=["riqueza"], topn=5))

[('perspicacia', 0.4525822401046753), ('infiernos', 0.38808712363243103), ('festín', 0.3599822521209717), ('paciencia', 0.3569086194038391), ('suelo', 0.3557209074497223)]


In [51]:
print(w2v_model.wv.most_similar(positive=["hombre", "virtud"], negative=["mujer"], topn=5))
print(w2v_model.wv.most_similar(positive=["razón", "conocimiento"], negative=["instinto"], topn=5))
print(w2v_model.wv.most_similar(positive=["arte", "belleza"], negative=["ciencia"], topn=5))
print(w2v_model.wv.most_similar(positive=["fuerza", "triunfo"], negative=["debilidad"], topn=5))
print(w2v_model.wv.most_similar(positive=["justicia", "ley"], negative=["corrupción"], topn=5))
print(w2v_model.wv.most_similar(positive=["amor", "pasión"], negative=["indiferencia"], topn=5))


[('malentendidos', 0.29007768630981445), ('ironía', 0.2850981652736664), ('moviliza', 0.2749691307544708), ('cruel', 0.2681245803833008), ('inclinación', 0.2626956105232239)]
[('indignación', 0.3698815405368805), ('platónico', 0.31873828172683716), ('dejas', 0.31352120637893677), ('coloca', 0.3114529848098755), ('virgen', 0.31016331911087036)]
[('plebeya', 0.3138872981071472), ('ordenó', 0.3088877499103546), ('derecha', 0.2994619905948639), ('participa', 0.2959606349468231), ('peores', 0.29321521520614624)]
[('exterior', 0.3562890291213989), ('aumento', 0.33026719093322754), ('reservados', 0.33005490899086), ('emperador', 0.3294997215270996), ('procurar', 0.31643372774124146)]
[('invertir', 0.33966776728630066), ('cometer', 0.32863566279411316), ('adversario', 0.32313141226768494), ('dignidad', 0.32178470492362976), ('fealdad', 0.3153897225856781)]
[('odio', 0.3619135022163391), ('federico', 0.35225746035575867), ('sometidos', 0.32844629883766174), ('filosofa', 0.3238115608692169), ('a

Los resultados pueden ser comprensibles en algunos casos, pero también pueden carecer de una relación semántica clara.

Por ejemplo, en la primera analogía, los términos más cercanos obtenidos incluyen "perspicacia", "infiernos" y "festín", lo que no parece una asociación evidente. Esto puede estar relacionado con la naturaleza del corpus utilizado en el entrenamiento, ya que podría haber captado conexiones indirectas o incluso sesgos en los datos.

Algunas analogías muestran patrones que coinciden con asociaciones comunes.

Por ejemplo, en la operación "justicia" + "ley" - "corrupción", las palabras más cercanas fueron "orden", "derecha" y "participa", lo que indica que el modelo reconoce en cierta medida conceptos ligados a la legalidad y el orden social. En otro caso, "razón" + "conocimiento" - "instinto" generó términos como "platónico", "indignación" y "dejas", lo que sugiere una posible relación con la filosofía, aunque algunos resultados no son los más esperados.

Sin embargo, ciertos resultados reflejan la presencia de ruido en el modelo o el corpus de entrenamiento.

Por ejemplo, en la analogía "amor" + "pasión" - "indiferencia", las palabras devueltas incluyen "filósofa", "ardiente" y "sometidos". Aunque algunas pueden relacionarse con la idea de pasión, no parecen ser las más precisas. Otro caso interesante es "hombre" + "virtud" - "mujer", donde aparecieron términos como "malentendidos", "ironía" y "cruel", lo que sugiere que el corpus puede contener sesgos en la representación de ciertos conceptos.

La calidad del corpus de entrenamiento es un factor determinante en los resultados.

Si el modelo se entrena con textos en los que las palabras tienen significados específicos dentro de un contexto determinado, es posible que aprenda asociaciones poco representativas o sesgadas. Un ejemplo de esto es que, si el corpus está compuesto en gran parte por textos filosóficos, el modelo tenderá a generar términos dentro de ese marco conceptual.

Por último, la analogía no siempre es una herramienta confiable para predecir relaciones semánticas.

Aunque los embeddings de palabras pueden capturar relaciones contextuales, los resultados no siempre se alinean con la lógica humana. En varios casos, se pueden encontrar asociaciones que tienen sentido hasta cierto punto, pero que también incluyen términos inesperados o poco relevantes.